In [ ]:
#Polar black and white parallel differential
import cv2
import numpy as np
img = cv2.imread('main_image_deep_field_smacs0723-5mb - Copy.jpg')
img = img.astype(np.float32)
scale = 1
max_radius = (np.sqrt(((img.shape[0] / 2.0) ** 2.0) + ((img.shape[1] / 2.0) ** 2.0))) / scale
polar_image = cv2.linearPolar(img,(img.shape[0] / 2, img.shape[1] / 2), max_radius, cv2.WARP_FILL_OUTLIERS)
polar_image = polar_image.astype(np.uint8)
cv2.imwrite('polart1.jpg', polar_image)
from PIL import Image
import matplotlib.pyplot as plt
#Read image
img=Image.open('polart1.jpg') 
img =img.transpose(Image.ROTATE_90)
#show image
plt.imshow(img)
plt.show()
img.size

In [ ]:
w, h = img.size
#instrumentName = 'harp'  #actual sound must be chosen after importing MIDI file into a DAW
#note_str=['X','C2', 'C3', 'G3', 'C4', 'E4', 'G4', 'A#4', 'C5', 'D5', 'E5', 'F#5', 'G5', 'G#5'] #first 13 notes of the harmonic series
#instrumentName = 'drums'
#note_str=['X','C2', 'D2', 'E2', 'F#2', 'G#2', 'A#2', 'G2', 'A2', 'C3', 'D#3', 'E3', 'C#3', 'A3'] #drum 'notes'
note_str=['C1', 'C2', 'G2', 'C3', 'E3', 'G3', 'A3', 'B3', 'D4', 'E4', 'G4', 'A4', 'B4',
         'D5', 'E5', 'G5', 'A5', 'B5','D6', 'E6', 'F#6', 'G6', 'A6',] #drum 'notes'
min_note=36
max_note=96
note_range = max_note-min_note ##len(note_str) 
small_img=img.resize((round(w/h*note_range),note_range),Image.BILINEAR)
plt.imshow(small_img)
small_img.size

In [ ]:
grey_img = small_img.convert('L')
plt.imshow(grey_img)

In [6]:
import numpy as np
pixels = np.array(grey_img, dtype=int)
#pixels.shape

In [7]:
differential = np.zeros(pixels.shape)
for j in range (1,round(w/h*note_range)):
    for i in range(0,note_range):
        differential[i,j]=abs(pixels[i,j]-pixels[i,j-1])

In [ ]:
plt.imshow(np.asarray(differential))
plt.show()
print(np.max(differential),np.min(differential),np.average(differential))

In [ ]:
duration_scale= 64
min_velocity = 0
max_velocity = 127
velocity_range = max_velocity- min_velocity
x = differential
grey_img2 = small_img.convert('L')
pixels2 = np.round(np.array(grey_img2, dtype=int) / duration_scale)
velocity = np.round(x / np.max(x) * velocity_range) + min_velocity
plt.imshow(np.asarray(velocity))

In [12]:
from midiutil import MIDIFile
from audiolazy import str2midi
bpm = 64
midifile = MIDIFile(adjust_origin=True)
midifile.addTempo(track=0, time=0, tempo=bpm)    
for j in range(0,round(w/h*note_range)):
    for i in range(0,note_range):#pitch=str2midi(note_str[i])
        if velocity[i,j] >= 30:
             midifile.addNote(track=0, channel=0, pitch=max_note-i, time=j, duration=pixels2[i,j]+1, volume=int(velocity[i,j]))
with open("pbawpd1.mid", "wb") as f:
    midifile.writeFile(f)

In [13]:
import cv2
import numpy as np
from PIL import Image
original_img= Image.open('main_image_deep_field_smacs0723-5mb - Copy.jpg')
width = height = 920
small_img = original_img.resize((width,height), Image.BILINEAR)
small_img.save('main_image_deep_field_smacs0723-5mb - Copy.png')
fourcc = 0
video = cv2.VideoWriter('video.mp4', fourcc, 1, (width, height))
img = cv2.imread('main_image_deep_field_smacs0723-5mb - Copy.png')
seconds = 31
for j in range(0,seconds):
    #img = cv2.imread('t1.jpg')
    video.write(img)
cv2.destroyAllWindows()
video.release()

In [14]:
from PIL import ImageDraw
cap = cv2.VideoCapture("video.mp4")
output = cv2.VideoWriter("moving.mp4",fourcc, 1, (width, height))
i = 0
while(True):
    ret, frame = cap.read()
    if(ret):
        i = i + 1
        # adding no fill rectangle on each frame
        #cv2.rectangle(frame, (int((i-1)/seconds*width),1), (int((i)/seconds*width),height-1),(255, 255, 255), 1)
        #adding transparent rectangle on each frame
        overlay = frame.copy()
        max_radius = np.sqrt(((overlay.shape[0] / 2.0) ** 2.0) + ((overlay.shape[1] / 2.0) ** 2.0))
        polar_image = cv2.linearPolar(overlay,(overlay.shape[0] / 2, overlay.shape[1] / 2), max_radius, cv2.WARP_FILL_OUTLIERS)
        polar_image = polar_image.astype(np.uint8)
        # Rectangle parameters
        x, y, w, h = 0,int((i-1)/seconds*height),width , int(1/seconds*height) 
        # A filled rectangle
        cv2.rectangle(polar_image, (x, y), (x+w, y+h), (255, 255, 255), -1)  
        alpha = 0.25  # Transparency factor.
        # Following line overlays transparent rectangle
        # over the image
        recovered_lin_polar_img = cv2.warpPolar(polar_image , (overlay.shape[0] , overlay.shape[1]),
                                            (overlay.shape[0] / 2, overlay.shape[1] / 2), max_radius,
                                            cv2.WARP_INVERSE_MAP);
        image_new = cv2.addWeighted(recovered_lin_polar_img, alpha, frame, 1 - alpha, 0)  

        
        # writing the new frame in output
        output.write(image_new)
        #cv2.imshow("output", frame)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break
    else:
        break
#print(i)
cv2.destroyAllWindows()
output.release()
cap.release()

In [15]:
import moviepy.editor as mpe
my_clip = mpe.VideoFileClip('moving.mp4')
audio_background = mpe.AudioFileClip('Sonification_of_first_image_of_NASAs_JWST_Polar_version.mp3')
final_clip = my_clip.set_audio(audio_background)
final_clip.write_videofile("output.mp4", fps = 30)

Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
